# Import des modules

In [1]:
import glob
import pandas as pd

# Lecture des données 

Données venant du Sirsé et cartosanté

In [2]:
liste = glob.glob('*.csv')
for i in range(len(liste)):
    if i == 0 :
        db = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
    else:
        db_temp = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
        try:
            db_temp.drop('Libellé',axis=1, inplace=True)
        except:
            pass
        db = db.merge(db_temp, left_on='Code', right_on='Code', how='inner')

Données de l'insee sur les codes de départements et regions

In [3]:
dep = pd.read_csv("../dep/communes-departement-region.csv", sep=",", encoding="utf-8")
#dep = dep.groupby(['code_commune_INSEE']).count().reset_index()
dep['code_commune_INSEE'] = dep['code_commune_INSEE'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_postal'] = dep['code_postal'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_dep'] = dep['code_commune_INSEE'].str[:2]
dep = dep[['code_dep','code_commune_INSEE', 
     'nom_departement','nom_region','code_postal',
     'code_region','longitude','latitude']]
db['PGP'] = db['Nbre de généralistes lib. au 31/12 2019']/db['Population municipale 2016']
db['PG16_>60'] = db['Nbre de généralistes lib. au 31/12 2019']/db['Nbre de généralistes lib. au 31/12 2019.1']
db['PC16_15_24'] = db['Nombre de chômeurs chez les 15-24 ans (RP) 2016']/db['Population municipale 2016']
db['PC16_24_64'] = db['Nombre de chômeurs chez les 15-64 ans (RP) 2016']/db['Population municipale 2016']
db['PPop16_<18'] = db['Population des moins de 18 ans 2016']/db['Population municipale 2016']
db['PPop16_>65'] = db['Population des 65 ans ou plus 2016']/db['Population municipale 2016']
db['PRP16'] = db['Nombre de résidences principales en HLM 2016']/db['Nombre de résidences principales 2016']
db.drop(["Nbre de généralistes lib. au 31/12 2019",
        "Nbre de généralistes lib. au 31/12 2019.1",
        "Population des moins de 18 ans 2016",
        "Nombre de chômeurs chez les 15-64 ans (RP) 2016",
        "Nombre de chômeurs chez les 15-24 ans (RP) 2016",
        "Population des 65 ans ou plus 2016",
        "Nombre de résidences principales 2016",
        "Nombre de résidences principales en HLM 2016"], axis=1, inplace=True )

# Modification des noms des colonnes

In [4]:
db.columns = ['Code','Nom_com', 'RFM16', 'TN16',
              'PM16','NSP19','DP16','PGP19','PGP19_>60',
              'PC16_15_24','PC16_24_64','PPop16_<18','PPop16_>65','PRP16']

In [5]:
db

,Code,Nom_com,RFM16,TN16,PM16,NSP19,DP16,PGP19,PGP19_>60,PC16_15_24,PC16_24_64,PPop16_<18,PPop16_>65,PRP16
0,01001,L'Abergement-Clémenciat,23440.0,9.1,767.0,0,49.0,0.000000,NaN,0.010430,0.043025,0.252934,0.186441,0.026144
1,01002,L'Abergement-de-Varey,24214.0,24.7,243.0,0,26.6,0.000000,NaN,0.016461,0.041152,0.255144,0.181070,0.000000
2,01004,Ambérieu-en-Bugey,19450.0,15.6,14081.0,1,575.0,0.000284,0.190476,0.018323,0.076699,0.239259,0.164903,0.205544
3,01005,Ambérieux-en-Dombes,23280.0,8.4,1671.0,0,104.1,0.000598,0.500000,0.014961,0.040694,0.241173,0.153800,0.048438
4,01006,Ambléon,NaN,18.2,110.0,0,18.2,0.000000,NaN,0.027273,0.072727,0.136364,0.236364,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34962,97613,M'Tsangamouji,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34963,97614,Ouangani,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34964,97615,Pamandzi,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34965,97616,Sada,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Jointure des données

In [6]:
merged_data = pd.merge(db,dep, left_on= "Code", right_on ="code_commune_INSEE", how="left")
data = pd.DataFrame(merged_data['Code'].value_counts() == 1)
liste_code_commune = data[data['Code']==True].index
merged_data_one = merged_data[merged_data['Code'].isin(liste_code_commune)]
#merged_data_one['code_region'] = merged_data_one['code_region'].astype(int).astype(str)
merged_data_one.drop(["Code","longitude","latitude"], axis=1, inplace=True)

C:\Users\joaka\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
merged_data_one.to_csv("output_data.csv")